# Feature Engineering Notebook
This notebook preprocesses data this project. 

**Note: this notebook loads data that is unavailable in csv form on github.** The raw complaint data has been saved as pkl files (as opposed to csv files) in the data_processed directory.

You can find the raw csv here: https://www.kaggle.com/kharaldsson/consumter-complaints?select=Consumer_Complaints_20190324.csv
You can find the csv with preprocessed data here: https://www.kaggle.com/kharaldsson/cfpb-feature-engineering/output?select=narr_df_02.csv


The data is used in in these notebooks:
* Random Forest Model: 
* SVM Bag-of-Words Model: 

In [1]:
import os
import time
from datetime import datetime, timedelta

import pandas as pd
import numpy as np
from numpy import copy
import re, string

import spacy
import en_core_web_md
nlp = en_core_web_md.load()

In [37]:
PATH_PARENT = os.path.dirname(os.getcwd())
PATH_RAW = PATH_PARENT + "\\data_raw\\"
PATH_PROC = PATH_PARENT + "\\data_processed\\"

# Functions

In [3]:
""" FUNCTIONS """


def clean_text(text):
    """
    Clean up the description: lowercase, remove brackets, punctuation
    """
    text = str(text)
    text = text.lower()
    text = re.sub(r'/', ' ', text)
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r"\s+", ' ', text).strip()
    return text


def lemmatizer(text):
    """
    Lemmatize & filter out stopwords in narrative
    """
    sent = []
    doc = nlp(text)
    for word in doc:
        if not word.is_stop:
            sent.append(word.lemma_)
    return " ".join(sent)

# Data Processesing

In [40]:
%%time

""" IMPORT DATA """
cc_raw_df = pd.read_csv(PATH_RAW+"Consumer_Complaints_20190324.csv")

<decorator-gen-55>:2: DtypeWarning: Columns (5,11,16) have mixed types.Specify dtype option on import or set low_memory=False.


Wall time: 5.96 s


In [5]:
cc_raw_df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,03/22/2019,Mortgage,Conventional home mortgage,Applying for a mortgage or refinancing an exis...,NaN,NaN,NaN,OCWEN LOAN SERVICING LLC,VA,221XX,NaN,NaN,Web,03/22/2019,Closed with explanation,Yes,NaN,3187532
1,03/22/2019,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,NaN,Company has responded to the consumer and the ...,USCB Corporation,MD,207XX,NaN,NaN,Web,03/22/2019,Closed with explanation,Yes,NaN,3188247
2,03/22/2019,Vehicle loan or lease,Loan,Struggling to pay your loan,Loan balance remaining after the vehicle is re...,NaN,Company has responded to the consumer and the ...,"First Investors Financial Services Group, Inc.",IL,NaN,NaN,NaN,Web,03/22/2019,Closed with explanation,Yes,NaN,3187579
3,03/22/2019,"Money transfer, virtual currency, or money ser...",International money transfer,Other transaction problem,NaN,NaN,NaN,RIA FINANCIAL,CA,90280,NaN,NaN,Phone,03/22/2019,In progress,Yes,NaN,3187502
4,03/22/2019,Debt collection,Other debt,Written notification about debt,Didn't receive enough information to verify debt,NaN,NaN,ALLY FINANCIAL INC.,FL,33162,NaN,NaN,Web,03/22/2019,In progress,Yes,NaN,3187649


In [6]:
cc_proc_df = cc_raw_df.copy()

# Clean up column titles
cc_proc_df.columns = cc_proc_df.columns.str.replace(' ', '_') 
cc_proc_df.columns = cc_proc_df.columns.str.replace('?', '', regex=False) 
cc_proc_df.columns = cc_proc_df.columns.str.replace('-', '_') 

# Modify column datatypes
cc_proc_df['Date_received'] = pd.to_datetime(cc_proc_df['Date_received'])
cc_proc_df['Date_sent_to_company'] = pd.to_datetime(cc_proc_df['Date_sent_to_company'])
cc_proc_df['Complaint_ID'] = cc_proc_df['Complaint_ID'].astype('int64')

# Remove Entries from Before Jan 2014
cc_proc_df = cc_proc_df[cc_proc_df['Date_received'] >= '2014-01-01']

# Remove In Progress Complaints
cc_proc_df = cc_proc_df[cc_proc_df['Company_response_to_consumer'] != "In progress"]


"""Create Additional Variables"""
# Administration Variables
cc_proc_df['Trump_Admin'] = np.where(cc_proc_df['Date_received'] >= '2017-01-20', 1, 0)
cc_proc_df['Mulvaney_Dir'] = np.where(cc_proc_df['Date_received'] >= '2017-11-25', 1, 0)

# Product Modifications Based on Changes to Tracking
cc_proc_df['New_Product'] = cc_proc_df['Product']
cc_proc_df['New_Product'] = np.where(cc_proc_df['New_Product'] == "Credit reporting"
                                     , "Credit reporting, credit repair services, or other personal consumer reports"
                                     , cc_proc_df['New_Product']
                                    )

cc_proc_df['New_Product'] = np.where((cc_proc_df['New_Product'] == "Credit card") 
                                     | (cc_proc_df['New_Product'] == "Prepaid card")
                                     , "Credit card or prepaid card"
                                     , cc_proc_df['New_Product']
                                    )

cc_proc_df['New_Product'] = np.where(cc_proc_df['New_Product'] == "Bank account or service"
                                     , "Checking or savings account"
                                     , cc_proc_df['New_Product']
                                    )

cc_proc_df['New_Product'] = np.where((cc_proc_df['Sub_product'] == "Vehicle loan") 
                                     | (cc_proc_df['Sub_product'] == "Vehicle lease")
                                     , "Vehicle loan or lease"
                                     , cc_proc_df['New_Product']
                                    )

cc_proc_df['New_Product'] = np.where((cc_proc_df['Sub_product'] == "Installment loan") 
                                     | (cc_proc_df['Sub_product'] == "Pawn loan")
                                     | (cc_proc_df['Sub_product'] == "Personal line of credit")
                                     | (cc_proc_df['Sub_product'] == "Title loan")
                                     | (cc_proc_df['Sub_product'] == "Payday loan")
                                     | (cc_proc_df['Product'] == "Payday loan")
                                     , "Payday loan, title loan, or personal loan"
                                     , cc_proc_df['New_Product']
                                    )

cc_proc_df['New_Product'] = np.where((cc_proc_df['Sub_product'] == "Check cashing") 
                                     | (cc_proc_df['Sub_product'] == "Debt settlement")
                                     | (cc_proc_df['Sub_product'] == "Domestic (US) money transfer")
                                     | (cc_proc_df['Sub_product'] == "Foreign currency exchange")
                                     | (cc_proc_df['Sub_product'] == "International money transfer")
                                     | (cc_proc_df['Sub_product'] == "Mobile wallet")
                                     | (cc_proc_df['Sub_product'] == "Money order")
                                     | (cc_proc_df['Sub_product'] == "Refund anticipation check")
                                     | (cc_proc_df['Sub_product'] == "Traveler’s / cashier’s checks")
                                     , "Money transfer, virtual currency, or money service"
                                     , cc_proc_df['New_Product']
                                    )



print(cc_proc_df.shape)
cc_proc_df.head()


(1054479, 21)


,Date_received,Product,Sub_product,Issue,Sub_issue,Consumer_complaint_narrative,Company_public_response,Company,State,ZIP_code,...,Consumer_consent_provided,Submitted_via,Date_sent_to_company,Company_response_to_consumer,Timely_response,Consumer_disputed,Complaint_ID,Trump_Admin,Mulvaney_Dir,New_Product
0,2019-03-22,Mortgage,Conventional home mortgage,Applying for a mortgage or refinancing an exis...,NaN,NaN,NaN,OCWEN LOAN SERVICING LLC,VA,221XX,...,NaN,Web,2019-03-22,Closed with explanation,Yes,NaN,3187532,1,1,Mortgage
1,2019-03-22,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,NaN,Company has responded to the consumer and the ...,USCB Corporation,MD,207XX,...,NaN,Web,2019-03-22,Closed with explanation,Yes,NaN,3188247,1,1,"Credit reporting, credit repair services, or o..."
2,2019-03-22,Vehicle loan or lease,Loan,Struggling to pay your loan,Loan balance remaining after the vehicle is re...,NaN,Company has responded to the consumer and the ...,"First Investors Financial Services Group, Inc.",IL,NaN,...,NaN,Web,2019-03-22,Closed with explanation,Yes,NaN,3187579,1,1,Vehicle loan or lease
7,2019-03-22,Debt collection,Other debt,Attempts to collect debt not owed,Debt is not yours,NaN,Company believes complaint represents an oppor...,"Hunter Warfield, Inc.",FL,337XX,...,NaN,Postal mail,2019-03-22,Closed with non-monetary relief,Yes,NaN,3187828,1,1,Debt collection
8,2019-03-22,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account information incorrect,NaN,NaN,OCWEN LOAN SERVICING LLC,FL,33014,...,NaN,Web,2019-03-22,Closed with explanation,Yes,NaN,3187550,1,1,"Credit reporting, credit repair services, or o..."


In [7]:
""" Write Full Processed Set to File"""

cc_proc_df.to_csv(PATH_PROC+"cc_proc_df_01.csv", index=False)

# Narrative Processing

In [8]:
nar_df = cc_proc_df.copy()

# Drop Narrativeless complaints
nar_df = nar_df.dropna(subset=['Consumer_complaint_narrative'])

print(nar_df.shape)
nar_df.head()

(374772, 21)


,Date_received,Product,Sub_product,Issue,Sub_issue,Consumer_complaint_narrative,Company_public_response,Company,State,ZIP_code,...,Consumer_consent_provided,Submitted_via,Date_sent_to_company,Company_response_to_consumer,Timely_response,Consumer_disputed,Complaint_ID,Trump_Admin,Mulvaney_Dir,New_Product
5746,2019-03-06,Debt collection,Credit card debt,False statements or representation,"Impersonated attorney, law enforcement, or gov...","On XXXX XXXX,2019 my friend got a call from XX...",Company has responded to the consumer and the ...,SYNCHRONY FINANCIAL,CA,NaN,...,Consent provided,Web,2019-03-07,Closed with explanation,Yes,NaN,3171169,1,1,Debt collection
6172,2019-03-05,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account information incorrect,There are many mistakes appear in my report wi...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,VA,220XX,...,Consent provided,Web,2019-03-05,Closed with explanation,Yes,NaN,3169888,1,1,"Credit reporting, credit repair services, or o..."
6298,2019-03-05,Debt collection,Private student loan debt,Attempts to collect debt not owed,Debt is not yours,I HAVE DISPUTED THIS ACCOUNT AND YOU HAVE FAIL...,Company has responded to the consumer and the ...,"Ability Recovery Services, LLC",OK,731XX,...,Consent provided,Web,2019-03-05,Closed with explanation,Yes,NaN,3170239,1,1,Debt collection
6984,2019-03-05,Debt collection,Other debt,Attempts to collect debt not owed,Debt was paid,This debit was paid however is has been on my ...,Company believes it acted appropriately as aut...,Source Receivables Management LLC,NY,NaN,...,Consent provided,Web,2019-03-05,Closed with explanation,Yes,NaN,3170244,1,1,Debt collection
7777,2019-03-04,Debt collection,Other debt,Written notification about debt,Didn't receive notice of right to dispute,I tried to apply for a car loan and discovered...,Company believes it acted appropriately as aut...,"Waypoint Resource Group, LLC",NM,871XX,...,Consent provided,Web,2019-03-08,Closed with explanation,Yes,NaN,3168736,1,1,Debt collection


In [9]:
"""Clean Up Narratives"""


# Lowercase, remove brackets, punctuation 
nar_df['narrative_clean'] = nar_df.Consumer_complaint_narrative.apply(lambda x: clean_text(x))

# Remove redaction
nar_df["narrative_clean"] = nar_df["narrative_clean"].str.replace('xxxx','')
nar_df["narrative_clean"] = nar_df["narrative_clean"].str.replace('xx','')
nar_df["narrative_clean"] = nar_df["narrative_clean"].str.replace('xxxxxxxx','')

# get word count
nar_df['nar_wordct'] = nar_df['narrative_clean'].str.split().str.len() 

# get character count
nar_df['nar_charct'] = nar_df['narrative_clean'].str.len() ## this also includes spaces

# get number of numerics
nar_df['nar_numerics'] = nar_df['narrative_clean'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))

In [10]:
%%time
"""Lemmatize"""
nar_df['narrative_lemma'] = nar_df.apply(lambda x: lemmatizer(x['narrative_clean']), axis=1)

Wall time: 2h 15min 10s


In [11]:
nar_df['narrative_clean_token'] = nar_df['narrative_clean'].apply(lambda x: str(x).split())
nar_df['narrative_lemma_token'] = nar_df['narrative_lemma'].apply(lambda x: str(x).split())

In [12]:
nar_df['company_clean'] = nar_df.Company.apply(lambda x: clean_text(x))

In [13]:
"""Write Preprocessed Data Out to File"""
# nar_df.to_csv(PATH_PROC+"narr_df_02.csv", index=False)

'Write Preprocessed Data Out to File'

In [16]:
"""
FUNCTIONS
"""

def createSpreadsheets(dataframe, version, chunk_size):
    if len(dataframe) >  chunk_size:
        remainder = len(dataframe)%chunk_size
        matters_create_remainder = dataframe.iloc[0:remainder+1]
        matters_create_chunkable = dataframe.iloc[remainder:]
    
        # Write matters_create_remainder to data_out
        matters_create_remainder.to_pickle(PATH_PROC+'/narr_df_'+version+'_00.pkl', compression='zip')
    
        # Write matters_Create_chunkable to data_out
        chunksize = chunk_size
        i = 1
        for chunk in np.split(matters_create_chunkable,len(matters_create_chunkable) // chunksize):
            chunk.to_pickle(PATH_PROC+'/narr_df_'+version+'_{:02d}.pkl'.format(i), compression='zip')
            i += 1
    else:
        # Write matters_create_remainder to data_out
        dataframe.to_pickle(PATH_PROC+'/narr_df_'+version+'_00.pkl', compression='zip')


In [17]:
createSpreadsheets(nar_df, '02', 30000)